# 图像的算术运算
## 图像加法
**OpenCV 相加操作和 Numpy 相加操作之间存在差异.** 
1. OpenCV 添加是饱和操作  
    饱和操作指的是当相加的结果超过像素值的上限（例如，在图像的像素值范围为0到255时，相加结果超过255），则将其截断为上限值。这意味着最终的像素值不会超过上限值，从而避免出现溢出的情况。
2. Numpy 添加是模运算  
    模运算是对相加结果进行模运算，将结果限制在像素值的范围内。例如，对于一个8位图像（像素值范围为0到255），相加结果大于255时，将对256取模，得到的余数作为最终的像素值。

In [ ]:
import cv2 as cv
import numpy as np
x = np.uint8([250])
y = np.uint8([10])
print(cv.add(x,y)) #250 + 10 =260 => 255
print(x + y)

## 图像混合

In [ ]:
import cv2 as cv
import numpy as np
img1= cv.imread('robot.png')
img2= cv.imread('opencv-logo-black.png')
img1 = cv.resize(img1, (150, 184))
print(img1.shape)
print(img2.shape)
dst = cv.addWeighted(img1,0.7,img2,0.3,0)
cv.imshow('dst',dst)
cv.waitKey(0)
cv.destroyAllWindows()


## 按位操作(融合图像)
这包括按位 AND，OR，NOT 和 XOR 运算。它们在提取图像的某一部分、定义和使用非矩形 ROI 等方面非常有用。

In [3]:
import cv2 as cv
import numpy as np
#加载两张图片
print(cv.__version__)
img1 = cv.imread('messi.jpg')
img2 = cv.imread('opencv-logo-black.png')

#我想在左上角放置一个logo，所以我创建了一个 ROI,并且这个ROI的宽和高为我想放置的logo的宽和高
rows,cols,channels = img2.shape
print(type(img2.shape))
roi = img1 [0:rows,0:cols]
#现在创建一个logo的掩码，通过对logo图像进行阈值，并对阈值结果并创建其反转掩码
img2gray = cv.cvtColor(img2,cv.COLOR_BGR2GRAY)
# 阈值处理
ret,mask = cv.threshold(img2gray,10,255,cv.THRESH_BINARY)
# 取反
mask_inv = cv.bitwise_not(mask)

#现在使 ROI 中的徽标区域变黑 ->获取背景
img1_bg = cv.bitwise_and(roi,roi,mask = mask_inv)
print((mask_inv.shape))
print((roi.shape))
 
#仅从徽标图像中获取徽标区域。 ->获取logo
print((img2.shape))
img2_fg = cv.bitwise_and(img2,img2,mask = mask)
print((img2_fg.shape))


#在 ROI 中放置徽标并修改主图像

dst = cv.add(img1_bg,img2_fg)
img1 [0:rows,0:cols] = dst
img_total = np.concatenate((img1_bg, img2_fg, dst), axis=1)
cv.imshow('img_total',img_total)
cv.waitKey(0)
cv.destroyAllWindows()



4.7.0
<class 'tuple'>
(184, 150)
(184, 150, 3)
(184, 150, 3)
(184, 150, 3)


```mermaid
    graph BT
        A(读取图片)-->B[背景图];
        A(读取图片)-->C[LOGO]-->J{与操作}
        
        B[背景图]-->|裁切 |D[ROI_logo]-->H{获取背景}
        B[背景图]-->|裁切 |E[others]-->N{拼接}
        
        C[LOGO] -->F[阈值处理]-->G[取反]--> H{与操作}-->I[背景]-->M["融合"]
        F[阈值处理]-->J{与操作}-->L[获取logo]-->M[融合]-->N{拼接}-->O[一张打上OpenCV logo的梅西]

        
        


```


###  阈值处理又称为二值化
使用 OpenCV 的滑动条来实现阈值的调节,并实时展示

In [ ]:
import cv2 as cv
import numpy as np

# 创建一个空白图像，用于显示调整后的阈值效果
img = cv.imread('messi.jpg')
cv.namedWindow('Threshold Demo')

# 定义滑动条的回调函数
def threshold_callback(value):
    threshold = value
    ret, thresholded_img = cv.threshold(img, threshold, 255, cv.THRESH_BINARY)
    cv.imshow('Threshold Demo', thresholded_img)

# 创建滑动条
cv.createTrackbar('Threshold', 'Threshold Demo', 0, 255, threshold_callback)

# 初始化图像显示
cv.imshow('Threshold Demo', img)

while True:
    # 按下 ESC 键退出程序
    if cv.waitKey(1) == 27:
        break

cv.destroyAllWindows()



```